In [23]:
import math
import numpy as np
import pandas as pd
import scipy as sp

PI = math.pi

PITCH = 0.55  # 螺距 55cm
SPEED = 1  # 头节点速率 1m/s
INIT_ANGLE = 16 * 2 * PI  # 初始角位置

NUM_OF_BODY_NODES = 223  # 不包括头节点
HEAD_BENCH_LEN = 3.41  # 龙头长 341cm
BODY_BENCH_LEN = 2.20  # 龙身板凳长 220cm
END_LEN = 0.275  # 孔中心距离近端 27.5cm

TOT_TIME = 300
TIMESTAMPS = [60 * i for i in range(6)]
KEY_NODES = [50 * i for i in range(5)] + [NUM_OF_BODY_NODES - 1]


def toCartesian(rad: float, ang: float) -> tuple[float, float]:
    return (rad * math.cos(ang), rad * math.sin(ang))


def toPolar(x: float, y: float) -> tuple[float, float]:
    return (math.hypot(x, y), math.atan2(y, x))


# 等距螺线表达式
def radialOf(ang: float) -> float:
    return PITCH * ang / (2 * PI)


# 单调递增的奇函数，非常漂亮
def arcLenOf(ang: float) -> float:
    return PITCH * (ang * math.sqrt(ang**2 + 1) + math.asinh(ang)) / (4 * PI)


TOT_ARC_LEN = arcLenOf(INIT_ANGLE)


def headNodeAngleOf(tm: float) -> float:
    eq = lambda theta: arcLenOf(theta) + SPEED * tm - TOT_ARC_LEN
    return sp.optimize.brentq(eq, 0.0, INIT_ANGLE, disp=True)


def angleOfNextNodeOf(cur: float, dist: float) -> float:
    eq = (
        lambda theta: PITCH**2
        * (theta**2 - 2 * cur * theta * math.cos(theta - cur) + cur**2)
        - 4 * PI**2 * dist
    )
    return sp.optimize.brentq(eq, cur, cur + PI, disp=True)

In [24]:
def positionsOfNodes(tm: float) -> list[tuple[float, float]]:
    results = []
    head_ang = headNodeAngleOf(tm)
    results.append(toCartesian(radialOf(head_ang), head_ang))
    ang = angleOfNextNodeOf(head_ang, HEAD_BENCH_LEN - 2 * END_LEN)
    for i in range(NUM_OF_BODY_NODES):
        results.append(toCartesian(radialOf(ang), ang))
        ang = angleOfNextNodeOf(ang, BODY_BENCH_LEN)
    return results


def solvePositions():
    node_names: list[str] = ["head x(m)", "head y(m)"]
    for i in range(NUM_OF_BODY_NODES):
        node_names.append(f"node {i} x(m)")
        node_names.append(f"node {i} y(m)")
    data: dict[str, np.ndarray] = {}
    for i in range(TOT_TIME + 1):
        data[f"{i} s"] = np.array(positionsOfNodes(float(i))).flatten()
    df = pd.DataFrame(data, index=node_names)
    df.reset_index(inplace=True)
    df.rename(columns={"index": "Node"}, inplace=True)
    df.to_csv("results1_positions.csv", index=False, float_format="%.6f")


solvePositions()